In [1]:
import folium
import ipywidgets as widgets
import math
from folium.map import Marker, Layer
from folium.features import PolyLine
from IPython.display import display, update_display, clear_output, Javascript
from enum import Enum
from collections import namedtuple
import requests
import geojson
from geojson import Feature, FeatureCollection, Polygon, Point
import json
from pprint import pprint

In [2]:
GeodexWebServiceAction = namedtuple('GeodexWebServiceAction', ['value', 'web_service_type', 'url_suffix'])

class GeodexWebServiceActions(Enum):
    TEXTINDEX_SEARCH = GeodexWebServiceAction(0, "GET", "textindex/search")
    TEXTINDEX_SEARCHSET = GeodexWebServiceAction(0, "GET", "textindex/searchset")
    SPATIAL_SEARCH_OBJECT = GeodexWebServiceAction(0, "GET", "spatial/search/object") 
    SPATIAL_SEARCH_RESOURCE = GeodexWebServiceAction(0, "GET", "spatial/search/resource")
    SPATIAL_SEARCH_RESOURCESET = GeodexWebServiceAction(0, "POST", "spatial/search/resourceset") 
    TYPEAHEAD_PROVIDERS = GeodexWebServiceAction(0, "GET", "typeahead/providers")
    GRAPH_RESDETAILS = GeodexWebServiceAction(0, "GET", "graph/resdetails")
    GRAPH_RESSETDETAILS = GeodexWebServiceAction(0, "POST", "graph/ressetdetails")
    GRAPH_RESSETPEOPLE = GeodexWebServiceAction(0, "POST", "graph/ressetpeople")

    def get_web_service_type(self):
        return self.value.web_service_type

    def get_url_suffix(self):
        return self.value.url_suffix

In [3]:
class GeodexWebServiceClient(object):
    
    def perform_web_service_call(self, geodex_web_service_action, data_dict, update_function):

        #Set core domain name for request URL
        domainURL = "https://geodex.org/api/v1/"

        #Create request URL
        requestURL = domainURL

        #Add the suffix for the call
        requestURL += geodex_web_service_action.get_url_suffix()

        #Get the type of request URL: GET or POST 
        web_service_type = geodex_web_service_action.get_web_service_type()
        
        #Send the request to the server
        if web_service_type=="POST":
            r = requests.get(requestURL, params=data_dict)
        elif web_service_type=="GET":
            r = requests.get(requestURL, params=data_dict)

        #Check status code and process data
        if r.status_code == requests.codes.ok:
            update_function(json.loads(json.dumps(r.json())))

In [4]:
class MainData(object):
    
    def __init__(self):
        self._spatial_results = []
        
    def process_spatial_results(self, data):
        self._spatial_results = []
        for row in data["features"]:
            feature = row
            url = row["properties"]["URL"]
            spatial_result = self._get_spatial_result(url)
            if spatial_result != None:
                spatial_result = self._get_spatial_result(url)
            else:
                spatial_result = SpatialResult(url)
                self._spatial_results.append(spatial_result)
            geometry = row["geometry"]["type"]
            coordinates = row["geometry"]["coordinates"]
            spatial_feature = SpatialFeature(feature, url, geometry, coordinates)
            spatial_result.add_spatial_feature(spatial_feature)
    
    def _get_spatial_result(self, url):
        for spatial_result in self._spatial_results:
            if spatial_result.get_url() == url:
                return spatial_result
        return None
        
    def process_details(self, data):
        s = data["S"];
        aname = data["Aname"];
        name = data["Name"];
        url = data["URL"];
        description = data["Description"];
        citation = data["Citation"];
        date_published = data["Datepublished"];
        curl = data["Curl"];
        keywords = data["Keywords"];
        license = data["License"];
        return Details(s, aname, name, url, description, citation, date_published, curl, keywords, license);
    
    def set_spatial_search_selector(self, spatial_search_selector):
        self._spatial_search_selector = spatial_search_selector
    
    def get_spatial_results(self):
        return self._spatial_results

In [5]:
class SpatialResult(object):
    
    def __init__(self, url):
        self._url = url
        self._spatial_features = []
      
    def get_spatial_features(self):
        return self._spatial_features
        
    def get_url(self):
        return self._url
    
    def add_spatial_feature(self, spatial_feature):
        self._spatial_features.append(spatial_feature)

In [6]:
class SpatialFeature(object):
    
    def __init__(self, feature, url, geometry, coordinates):
        self._feature = feature
        self._url = url
        self._geometry = geometry
        self._coordinates = self._process_coordinates(coordinates)
        
    def _process_coordinates(self, coordinates):
        coordinate_array = []
        coordinates_str = ""
        if self._geometry == "Polygon":
            coordinates_str = ''.join(str(v) for v in coordinates)
            coordinates_str = coordinates_str[1:-1]
        elif  self._geometry == "Point":
            coordinates_str = ','.join(str(v) for v in coordinates)
        coordinates_str = coordinates_str.replace("[", "")
        coordinates_str = coordinates_str.replace("]", "")
        array = str(coordinates_str).split(",")
        for i in range(0, len(array)-1, 2):
            coordinate_array.append([array[i].strip(), array[i+1].strip()]);
        return coordinate_array;
    
    def get_feature(self):
        return self._feature
        
    def get_url(self):
        return self._url
    
    def get_geometry(self):
        return self._geometry
    
    def get_coordinates(self):
        return self._coordinates
    
    def get_coordinates_as_string(self):
        string = ""
        length = 0
        if self._geometry == "Point":
            length = len(self._coordinates)
        elif self._geometry == "Polygon":
            length = len(self._coordinates)-1
        for i in range(length):
            lat = self._coordinates[i][1]
            lon = self._coordinates[i][0]
            string += "[" + lat + ", " + lon + "]"
            if i < (len(self._coordinates)-2):
                string += ", "
        return string;
    
    def get_coordinates_as_array(self):
        coordinate_array = []
        for i in range(0, len(self._coordinates)-1):
            coordinate_array.append([self._coordinates[i][0], self._coordinates[i][1]])
        return coordinate_array

In [7]:
class Details(object):
    
    def __init__(self, s, aname, name, url, description, citation, date_published, curl, keywords, license):
        self._s = s
        self._aname = aname
        self._name = name
        self._url = url
        self._description = description
        self._citation = citation
        self._date_published = date_published
        self._curl = curl
        self._keywords = keywords
        self._license = license
        
    def get_s(self):
        return self._s
        
    def get_aname(self):
        return self._aname
    
    def get_name(self):
        return self._name
    
    def get_url(self):
        return self._url
    
    def get_description(self):
        return self._description
    
    def get_citation(self):
        return self._citation
    
    def get_date_published(self):
        return self._date_published
    
    def get_curl(self):
        return self._curl
    
    def get_keywords(self):
        return self._keywords
    
    def get_license(self):
        return self._license

In [8]:
class SpatialSearchSelector(object):

    def __init__(self, geodex_web_service_client, updatable_map_selector, main_data, spatial_search_viewer):
        self._updatable_map_selector = updatable_map_selector
        self._geodex_web_service_client = geodex_web_service_client
        self._main_data = main_data
        self._spatial_search_viewer = spatial_search_viewer
        self._current_location = self._updatable_map_selector.get_current_location()
        self._layout_ui()
        self._initialize_ui()
        self.show_ui()
    
    def show_ui(self):
        self._updatable_map_selector.show_map()
        display(self._container)
    
    def hide_ui(self):
        clear_output()
    
    def _on_submit_button_clicked(self, button):
        feature_collection_str = self._get_feature_collection_str()
        data_dict = {'geowithin':feature_collection_str}
        self._geodex_web_service_client.perform_web_service_call(GeodexWebServiceActions.SPATIAL_SEARCH_OBJECT, 
                                                                 data_dict, 
                                                                 self.update_after_spatial_search_object)
    
    def _get_feature_collection_str(self):
        sorted_locations = self._updatable_map_selector.get_sorted_locations()
        features = []
        for location in sorted_locations:
            point = Point([location[1], location[0]]) 
            feature = Feature(geometry=point)
            features.append(feature)
        feature_collection = FeatureCollection(features)
        feature_collection_str = geojson.dumps(feature_collection, sort_keys=True)
        return feature_collection_str;
        
    def update_after_spatial_search_object(self, data):
        self._main_data.process_spatial_results(data)
        self._spatial_search_viewer.hide_ui()
        self._spatial_search_viewer.initialize_ui(self._updatable_map_selector.get_sorted_locations())
        self._spatial_search_viewer.show_ui()
    
    def _on_update_button_clicked(self, button):
        self._updatable_map_selector.set_current_location([float(self._lat_box.value), float(self._lon_box.value)], True)

    def _on_add_button_clicked(self, button):
        self._updatable_map_selector.set_current_location([float(self._lat_box.value), float(self._lon_box.value)])
        new_option = self._lat_box.value + ", " + self._lon_box.value
        if new_option not in self._location_select.options:
            self._location_select.options = self._location_select.options + (new_option,)
            self._location_select.value = new_option
            self._updatable_map_selector.add_saved_location([float(self._lat_box.value), float(self._lon_box.value)])
            self._updatable_map_selector.set_current_location([float(self._lat_box.value), float(self._lon_box.value)])
    
    def _on_remove_button_clicked(self, button):
        self._updatable_map_selector.set_current_location([float(self._lat_box.value), float(self._lon_box.value)])
        old_options_list = list(self._location_select.options)
        
        if self._location_select.value is not None:
            old_options_list.remove(self._location_select.value)
            
            location_string_array = self._location_select.value.split(",")
            
            self._updatable_map.remove_saved_location([float(location_string_array[0].strip()), 
                                                        float(location_string_array[1].strip())])
            
            if old_options_list:
                self._location_select.options = tuple(old_options_list)
                self._location_select.value = self._location_select.options[0]
                
            else:
                self._location_select.options = ()
                self._location_select.value = None
      
    def _initialize_ui(self):
        self._lat_box.value = str(self._current_location[0])
        self._lon_box.value = str(self._current_location[1])
    
    def _layout_ui(self):
        
        #Labels
        self._lat_label = widgets.Label(
            value = 'Current Latitude: ', 
            layout = widgets.Layout(
                width = '35%', 
                margin = '5px 5px 0px 5px'
            )
        )
        self._lon_label = widgets.Label(
            value='Current Longtitude: ', 
            layout = widgets.Layout(
                width = '35%', 
                margin = '5px 5px 0px 5px'
            )
        )
        self._location_label = widgets.Label(
            value='List of Selected Locations: ', 
            layout = widgets.Layout(
                width = '100%', 
                margin = '5px 5px 0px 5px'
            )
        )
        
        #Textfields
        self._lat_box = widgets.Text(
            layout = widgets.Layout(
                width = '90%', 
                margin = '0px 5px 0px 5px'
            )
        )
        self._lon_box = widgets.Text(
            layout = widgets.Layout(
                width = '90%', 
                margin = '0px 5px 0px 5px'
            )
        )
        
        #Buttons
        self._update_button = widgets.Button(
            description = 'Update Location on Map',
            layout = widgets.Layout(
                width = '33%', 
                margin = '15px 5px 0px 5px'
            )
        )
        self._update_button.on_click(self._on_update_button_clicked)
        
        self._add_button = widgets.Button(
            description = 'Add Location to List',
            layout = widgets.Layout(
                width = '33%', 
                margin = '15px 5px 5px 5px'
            )
        )
        self._add_button.on_click(self._on_add_button_clicked)
        
        self._remove_button = widgets.Button(
            description = 'Remove Location from List',
            layout = widgets.Layout(
                width = '34%', 
                margin = '15px 5px 5px 5px'
            )
        )
        self._remove_button.on_click(self._on_remove_button_clicked)
        
        self._submit_button = widgets.Button(
            description = 'Submit Search to EarthCube.org',
            layout = widgets.Layout(
                width = '99%', 
                margin = '5px 5px 5px 5px'
                
            )
        )
        self._submit_button.on_click(self._on_submit_button_clicked)
        
        #Selects
        self._location_select = widgets.Select(
            layout = widgets.Layout(
                width = '100%', 
                height = '100%', 
                margin = '5px 0px 5px 0px'
            )
        )
        
        #Containers
        self._location_container = widgets.VBox(
            [self._lat_label, 
             self._lat_box, 
             self._lon_label, 
             self._lon_box]
        )
        self._location_container.layout.width = '50%'
        
        self._location_select_container = widgets.VBox(
            [self._location_label, 
             self._location_select]
        )
        self._location_select_container.layout.width = '50%'
        
        self._middle_container = widgets.HBox(
            [self._location_container, 
             self._location_select_container]
        )
        self._middle_container.layout.width = '100%'
        
        self._bottom_container = widgets.HBox(
            [self._update_button, 
             self._add_button, 
             self._remove_button]
        )
        
        self._container = widgets.VBox(
            [self._middle_container, 
             self._bottom_container, 
             self._submit_button]
        )

In [9]:
class UpdatableMapSelector(object):
    
    def __init__(self):
        
        #BOULDER!
        self._current_location = [40.0150, -105.2705]
        self._saved_locations = []
        self._sorted_locations = []
        self._old_markers = []
        self._old_lines = []
        
        self._map = folium.Map(self._current_location, max_bounds=True)
    
        self._current_marker = Marker(location=self._map.location, 
                                      popup=folium.Popup(str(self._map.location[0]) 
                                                         + ", " 
                                                         + str(self._map.location[1]))
                                     , icon=folium.Icon(color='green'))
        self._current_marker.add_to(self._map)
    
    def show_map(self):
        display(self._map, display_id="updatable_map_selector")
        
    def get_sorted_locations(self):
        return self._sorted_locations;
        
    def set_current_location(self, location, redraw_map=False):
        self._current_location = location
        if redraw_map:
            self._redraw_map()
    
    def add_saved_location(self, location):
        self._saved_locations.append(location)
        self._redraw_map()
    
    def remove_saved_location(self, location):
        self._saved_locations.remove(location)
        self._redraw_map()
    
    def get_current_location(self):
        return self._current_location
        
    def get_map(self):
        return self._map
    
    def get_saved_locations(self):
        return self._saved_locations
    
    def _redraw_map(self):
        
        self._sorted_locations = []
        
        #We can't remove the old markers or lines using Folium, 
        #but we can break it and assign non-real coordinates and
        #make them dissappear into space
        for marker in self._old_markers:
            marker.location = [1000, 1000]
        for line in self._old_lines:
            line.location = [[1000, 1000], [1000, 1000]]

        self._current_marker.location = self._current_location
        self._current_marker.popup = folium.Popup(str(self._current_location[0]) 
                                                         + ", " 
                                                         + str(self._current_location[1]))
            
        for saved_location in self._saved_locations:
            marker = Marker(location=saved_location, 
                                      popup=folium.Popup(str(saved_location[0]) 
                                                         + ", " 
                                                         + str(saved_location[1])))
            marker.add_to(self._map)
            self._old_markers.append(marker)
            self._sorted_locations.append(saved_location)

        if len(self._saved_locations) == 2:
            line = PolyLine(self._saved_locations)
            line.add_to(self._map)
            self._old_lines.append(line)
            self._sorted_locations = self._saved_locations
        
        if len(self._saved_locations) > 2:
            self._sorted_locations = self._sort_locations(self._saved_locations)
            self._sorted_locations.append(self._sorted_locations[0])
            line = PolyLine(self._sorted_locations)
            line.add_to(self._map)
            self._old_lines.append(line)

        self._set_fit_bounds()
        update_display(self._map, display_id="updatable_map_selector")
        display(Javascript("window.scrollTo(0,document.body.scrollHeight);")) 
        
    def _set_fit_bounds(self):
        
        ne_location_lat = self._current_location[0]
        ne_location_long = self._current_location[1]
        sw_location_lat = self._current_location[0]
        sw_location_long = self._current_location[1]
        
        for saved_location in self._saved_locations:   
            if saved_location[0] < sw_location_lat:
                sw_location_lat = saved_location[0]
            if saved_location[1] < sw_location_long:
                sw_location_long = saved_location[1]
           
        for saved_location in self._saved_locations:
            if saved_location[0] > ne_location_lat:
                ne_location_lat = saved_location[0]
            if saved_location[1] > ne_location_long:
                ne_location_long = saved_location[1]
                
        sw_location = [sw_location_lat, sw_location_long]
        ne_location = [ne_location_lat, ne_location_long]

        self._map.fit_bounds([sw_location, ne_location])    
        
    def _polar_sort(self, locations):
        lat, lon = zip(*((location[0], location[1]) for location in locations))
        ave_lat = float(sum(lat))/len(lat)
        ave_lon = float(sum(lon))/len(lon)
        return sorted(locations, key=lambda location: math.atan2(location[0]-ave_lat, location[1]-ave_lon))
        
    def _sort_locations(self, locations):
        lat,lon = zip(*((location[0], location[1]) for location in self._polar_sort(locations)))
        sorted_locations = []
        for i in range(0, len(lat)):
            sorted_locations.append([lat[i], lon[i]])
        return sorted_location

In [10]:
class SpatialSearchViewer(object):
    
    def __init__(self, geodex_web_service_client, main_data, updatable_map_viewer):
        self._geodex_web_service_client = geodex_web_service_client
        self._main_data = main_data
        self._updatable_map_viewer = updatable_map_viewer
        self._result_counter = 0
        self._layout_ui()
        
    def set_spatial_search_selector(self, spatial_search_selector):
        self._spatial_search_selector = spatial_search_selector
    
    def show_ui(self):
        self._updatable_map_viewer.redraw_map()
        display(self._container)
        
    def hide_ui(self):
        clear_output()
    
    def initialize_ui(self, sorted_locations):
        self._updatable_map_viewer.set_current_spatial_result(self._main_data.get_spatial_results()[self._result_counter])
        self._updatable_map_viewer.set_sorted_locations(sorted_locations)
        self._previous_button.disabled=True
        self._update_results_label()
        
    def _update_results_label(self):
        spatial_result = self._main_data.get_spatial_results()[self._result_counter]
        url = spatial_result.get_url()
        spatial_features = spatial_result.get_spatial_features()

        html = '<table cellpadding="5">' 
        html+= '<tr><td><b>Result Number ' 
        html+= str(self._result_counter + 1) 
        html+= " out of " 
        html+= str(len(self._main_data.get_spatial_results())) 
        html+= '</b></td></tr>' 
        html+= '<tr><td>' 
        html+= '<b><a target="_blank" href="' + url + '">' + url + "</a></b>"
        html+= '</td></tr>' 
        html+= '<tr><td><b>Features:</b></td></tr>'
        
        for spatial_feature in spatial_features:
            geometry = spatial_feature.get_geometry()
            coordinates = spatial_feature.get_coordinates_as_string()
            html+= '<tr><td>Geometry - ' 
            html+= geometry
            html+= ', Coordinates - ' 
            html+= coordinates
            html+= '</td></tr>'
            
        html+= '</table>';
        self._results_label.value = html
        
    def _on_previous_button_clicked(self, button):
        clear_output()
        self._next_button.disabled=False
        if self._result_counter > 0:
            self._result_counter = self._result_counter - 1 
        self._updatable_map_viewer.set_current_spatial_result(self._main_data.get_spatial_results()[self._result_counter])
        self._update_results_label()
        if self._result_counter == 0:
            self._previous_button.disabled=True
        self._updatable_map_viewer.redraw_map()
        display(self._container)
    
        js = """window.scrollTo(0,document.body.scrollHeight);"""
        display(Javascript(js))
        
    def _on_next_button_clicked(self, button):
        clear_output()
        self._previous_button.disabled=False
        if self._result_counter < (len(self._main_data.get_spatial_results())-1):
            self._result_counter = self._result_counter + 1 
        self._updatable_map_viewer.set_current_spatial_result(self._main_data.get_spatial_results()[self._result_counter])
        self._update_results_label()
        if self._result_counter == (len(self._main_data.get_spatial_results())-1):
            self._next_button.disabled=True
        self._updatable_map_viewer.redraw_map()
        display(self._container)
        
    def _on_back_button_clicked(self, button):
        self.hide_ui()
        self._spatial_search_selector.show_ui()
    
    def _layout_ui(self):  
        
        #Buttons
        self._back_button = widgets.Button(
            description = 'Modify Spatial Search at EarthCube.org',
            layout = widgets.Layout(
                width = '99%', 
                margin = '5px 5px 5px 5px'
            )
        )
        self._back_button.on_click(self._on_back_button_clicked)
        
        self._previous_button = widgets.Button(
            description = '< Previous Result',
            layout = widgets.Layout(
                width = '20%', 
                margin = '5px 5px 5px 5px'
            )
        )
        self._previous_button.on_click(self._on_previous_button_clicked)
        
        self._next_button = widgets.Button(
            description = 'Next Result >',
            layout = widgets.Layout(
                width = '20%', 
                margin = '5px 5px 5px 5px'
            )
        )
        self._next_button.on_click(self._on_next_button_clicked)
        
        self._results_label = widgets.HTML(
            value=""
        )
        
        self._top_container = widgets.HBox(
            [self._previous_button, 
                self._results_label, 
                self._next_button]
        )
        
        self._container = widgets.VBox(
            [self._top_container, 
             self._back_button]
        )

In [11]:
class UpdatableMapViewer(object):
    
    def __init__(self, ):
        
        self._current_location = [40.0150, -105.2705]
        self._sorted_locations = []
        self._old_markers = []
        self._old_lines = []
        self._current_spatial_result = None
        self._map = folium.Map(self._current_location, max_bounds=True)
        self._geojson_name_counter = 0

    def set_sorted_locations(self, sorted_locations):
        self._sorted_locations = sorted_locations
    
    def redraw_map(self):

        self._map = folium.Map(self._current_location, max_bounds=True)

        #We can't remove the old markers or lines using Folium, 
        #but we can break it and assign non-real coordinates and
        #make them dissappear into space
        for marker in self._old_markers:
            marker.location = [1000, 1000]
        for line in self._old_lines:
            line.location = [[1000, 1000], [1000, 1000]]

        for sorted_location in self._sorted_locations:
            marker = Marker(location=sorted_location, 
                                      popup=folium.Popup(str(sorted_location[0]) 
                                                         + ", " 
                                                         + str(sorted_location[1])))
            marker.add_to(self._map)
            self._old_markers.append(marker)

        if len(self._sorted_locations) == 2:
            line = PolyLine(self._sorted_locations)
            line.add_to(self._map)
            self._old_lines.append(line)
        
        if len(self._sorted_locations) > 2:
            line = PolyLine(self._sorted_locations)
            line.add_to(self._map)
            self._old_lines.append(line)

        spatial_result = self._current_spatial_result
        spatial_features = spatial_result.get_spatial_features()
        
        for spatial_feature in spatial_features:
            self._geojson_name_counter = self._geojson_name_counter + 1
            layer_name = 'geojson' + str(self._geojson_name_counter)
            geojson_layer = folium.GeoJson(spatial_feature.get_feature(), name=layer_name).add_to(self._map) 

        self._set_fit_bounds()
        display(self._map, display_id="updatable_map_viewer")
        
    def _set_fit_bounds(self):
        
        ne_location_lat = self._current_location[0]
        ne_location_long = self._current_location[1]
        sw_location_lat = self._current_location[0]
        sw_location_long = self._current_location[1]
        
        for sorted_location in self._sorted_locations:   
            if sorted_location[0] < sw_location_lat:
                sw_location_lat = sorted_location[0]
            if sorted_location[1] < sw_location_long:
                sw_location_long = sorted_location[1]
           
        for sorted_location in self._sorted_locations:
            if sorted_location[0] > ne_location_lat:
                ne_location_lat = sorted_location[0]
            if sorted_location[1] > ne_location_long:
                ne_location_long = sorted_location[1]
        
        spatial_result = self._current_spatial_result
        spatial_features = spatial_result.get_spatial_features()
        
        for spatial_feature in spatial_features:
            
            coordinates = spatial_feature.get_coordinates_as_array()
        
            for coordinate in coordinates:

                lat = float(coordinate[1])
                lon = float(coordinate[0])

                if lat < sw_location_lat:
                    sw_location_lat = lat
                if lon < sw_location_long:
                    sw_location_long = lon
                if lat > ne_location_lat:
                    ne_location_lat = lat
                if lon > ne_location_long:
                    ne_location_long = lon

        sw_location = [sw_location_lat, sw_location_long]
        ne_location = [ne_location_lat, ne_location_long]

        self._map.fit_bounds([sw_location, ne_location])    
        
    def set_current_spatial_result(self, spatial_result):
        self._current_spatial_result = spatial_result


In [12]:
geodex_web_service_client = GeodexWebServiceClient()
updatable_map_selector = UpdatableMapSelector()
updatable_map_viewer = UpdatableMapViewer()
main_data = MainData()
spatial_search_viewer = SpatialSearchViewer(geodex_web_service_client, main_data, updatable_map_viewer)
spatial_search_selector = SpatialSearchSelector(geodex_web_service_client, updatable_map_selector, 
                                                main_data, spatial_search_viewer)
spatial_search_viewer.set_spatial_search_selector(spatial_search_selector)
main_data.set_spatial_search_selector(spatial_search_selector)

A Jupyter Widget